# [1] モデルをw2 * t_u ** 2 + w1 * t_u + bとして再定義してください
※ ここではGoogle Colaraboratoryでの実行を想定しています。

※ Google Colaraboratoryでbashコマンドを実行するには、命令の前に!をつけます。

### （a）再定義を行うために、トレーニングループのどの部分を変更する必要がありますか？

In [1]:
# 回答

In [2]:
# モデルの定義部分とモデルパラメータの定義部分

In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0,
                    8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [3]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [4]:
# モデルの定義を変更
def model(t_u, w1,w2, b):    
    return w2 * t_u ** 2 + w1 * t_u +b

In [5]:
# モデルパラメータの定義を変更
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-3
optimizer = optim.SGD([params], lr=learning_rate)

In [6]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]


train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [10]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)

        with torch.no_grad(): # <1>
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad == False 
            
        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")
            
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

In [12]:
training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un, # <1> 
    val_t_u = val_t_un, # <1> 
    train_t_c = train_t_c,
    val_t_c = val_t_c)

# 注意：本セルの結果は、実行のたびに変化し、ときに学習が失敗します。

Epoch 1, Training loss nan, Validation loss nan
Epoch 2, Training loss nan, Validation loss nan
Epoch 3, Training loss nan, Validation loss nan
Epoch 500, Training loss nan, Validation loss nan
Epoch 1000, Training loss nan, Validation loss nan
Epoch 1500, Training loss nan, Validation loss nan
Epoch 2000, Training loss nan, Validation loss nan
Epoch 2500, Training loss nan, Validation loss nan
Epoch 3000, Training loss nan, Validation loss nan


### （b）モデルが変更されたことを知り得ない部分はどこでしょうか？

In [ ]:
# 回答

In [35]:
# 例：（データセットの分割、前処理）とくに、lossの計算

### （c）訓練後にロスは高くなりますか、それとも低くなりますか？

In [36]:
# 回答

In [37]:
# 低くなる。訓練とは損失関数の値（loss）を低下させるプロセスである。

###（d）結果は、本章の内容より良くなりますか、それとも悪くなりますか？

In [38]:
# 回答

In [ ]:
# 検証データにおいて性能が低く、モデルの学習結果は悪くなる
# 華氏から摂氏からへの変換式は、線形モデル：Tc = 0.56*Tf -17.78
# のため、より複雑なモデル（今回は2乗項を含むモデル）でフィッティングした結果、過学習が発生している。

# 本ノートブックの結果： Epoch 3000, Training loss 2.4431, Validation loss 20.3329 
# 本章の結果：Epoch 3000, Training loss 3.0125, Validation loss 3.5756
# 本演習では、2乗項を含むため、訓練データにはフィットしやすく、訓練データでのlossは小さくなったが、過学習が発生し、検証データでのlossは増加していった